In [3]:
# Link to GitLab Repository
GitRep = "https://gitlab.gbar.dtu.dk/s224222/computational_assignment1"


In [1]:
import requests
from bs4 import BeautifulSoup
import difflib

WEEK 1 - WEBSCRABING & API

Webscraping

In [ ]:
url = "https://ic2s2-2023.org/program"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

In [ ]:
soup.find_all("i")

In [ ]:
# Get all the names into a table
names = []
for i in soup.find_all("i"):
    names.append(i.text)

names = ",".join(names)


# if "chair" in names: Remove the "chair"

names = names.replace("Chair: ", "")
names = names.split(",")
map(str.strip, names)
print(names)

In [ ]:
# Remove doublicates
for i in range(len(names)):
    names[i] = names[i].lower()
    names[i] = names[i].strip()
names = list(set(names))
# and sort alphabetically

names.sort()
print(names)

In [ ]:
duplicate = []
for i in range(len(names)):
    for j in range(i+1, len(names)):
        if difflib.SequenceMatcher(None, names[i], names[j]).ratio() > 0.95:
            duplicate.append(names[j].lower().strip())

print(duplicate)

In [ ]:
# Remove one of the duplicates from the list
names = [x for x in names if x not in duplicate]
print(names)

In [ ]:
print(len(names)), print(names)

In [ ]:
# Create a txt file with the names

with open("names.txt", "w") as f:
    for name in names:
        f.write(name + "\n")